First of all, let's download the file. Thankfully wheelsaroundme allows public access to his Google Sheet spreadsheet, so we don't need to create and API key yada-yada-yada

In [70]:
from pathlib import Path

import pandas as pd

if Path("wam.csv").exists():
    df = pd.read_csv("wam.csv")
else:
    url = "https://docs.google.com/spreadsheets/d/1ttSU56TcDoZBzcCAIp7VPEjHDIcIuyi8wtb5FO3V2bA/export?format=csv"
    df = pd.read_csv(url)
    df.to_csv('wam.csv')
df

,Unnamed: 0,Jenis Kendaraan,Tahun,CC mesin,GEARBOX,Konsumsi dalam kota (KM/L),Konsumsi luar kota (KM/L),Jenis Bensin,USERNAME/NAMA (demi validitas),DOMISILI,...,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27
0,0,Audi A4 B8 1.8 TFSI,2011,1800.0,CVT,8,12,Pertamax,iamdiasxz,Bojonegoro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Audi A6 C6 2.0 TFSI,2011,2000.0,CVT,8-9,12-13,Vivo 95,Kevin.taruna,Tangerang Selatan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Austin Cooper S MK1,1965,1293.0,4MT,5-7,10,Shell Vpower/ Pertamax Turbo,Anggarazk,Bekasi,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,BMW 120i E87,2005,2000.0,6 speed AT,9-10,15,Pertamax,Santoso97,Jakarta,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,BMW 218i Active Tourer F45,2015,1500.0,A/T 6 speed,12,16,Pertamax Turbo,dvn,Tangerang Selatan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,1128,Mercedes-Benz 300E (W124),1989,3000.0,M/T 5 speed,4-5,6-7,Pertamax 92,chaskapetrolhead,Jakarta,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1129,1129,BMW 520i (E34) (stroke up to 2500cc),1991,2500.0,M/T 5 speed,5,7,Pertamax 92,chaskapetrolhead,Jakarta,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1130,1130,Hyundai Trajet AT GL8 non CVVT,2005,2000.0,AT 4-Speed,6-7,8-9,Pertamax 92,_da.vin,Jakarta,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1131,1131,Mitsubishi Galant ST V6 2.500CC,2000,2500.0,AT 4-speed,6-8,10-11,Pertalite - Pertamax,agungiki,Bekasi-Tangerang,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


There are some problems with the data:
1. Some users put the "CC mesin" (engine cc) column in liters instaed
2. Gearbox values are wild
3. One user put "A" in the "Tahun" (model year) column 
4. "Konsumsi dalam kota (KM/L)" (city fuel consumption in km/L) and "Konsumsi luar kota (KM/L)" (highway fuel consumption in km/L) values are also wild

Let's deal with the issues one-by-one

Since we know for a fact that 1L = 1000cc, we could just normalize them together. Here we assumed no car engine displacement is less than 100cc nor more than 100L, for obvious reason.


In [71]:
df.loc[:, ['Jenis Kendaraan', 'CC mesin']].sort_values("CC mesin").head()

,Jenis Kendaraan,CC mesin
77,Daihatsu Ayla X 1.0 MT,1.0
863,Toyota Raize 1.0 T GR,1.0
75,Daihatsu Ayla M 1.0 MT,1.0
663,Suzuki Ignis AGS,1.2
664,Suzuki Ignis AGS,1.2


In [72]:
df.loc[df['CC mesin'] < 100, 'CC mesin'] *= 1000
df.loc[:, ['Jenis Kendaraan', 'CC mesin']].sort_values("CC mesin").head()

,Jenis Kendaraan,CC mesin
80,Daihatsu Ceria KX,849.0
676,Suzuki Karimun Kotak SL410R,985.0
337,KIA Morning 1.0 MT,1000.0
1048,Toyota Agya 1.0 TRDs,1000.0
1052,Daihatsu Ayla 1.0 X,1000.0


In [73]:
df["GEARBOX"].unique()

array(['CVT', '4MT', '6 speed AT', 'A/T 6 speed', 'AT', '5MT', 'MT',
       'AT Dual Clutch', 'ZF 8HP 8-speed AT', '8AT', 'At',
       'AT Tiptronic 5', '8 Speed AT', 'M/T', 'AT 5 speed', 'MT 5 speed',
       'AT Stepteonic', 'AT 6 speed', '6A/T ZF6HP19', 'A/T 6 Speed',
       'AT 8 Speed', 'M/T 5 speed', 'M/T 5 Speed', '6AT', 'AT 6 Speed',
       'MT 5speed', '5 M/T', 'manual', 'AT 4 Speed', 'Manual',
       'AT 4 Speed Tiptronic', 'MT 4 Speed', 'AT (Dual Clutch)',
       'Dual Clutch AT', 'MT 5 Speed', 'A/T', '5 MT', 'Cvt',
       'Manual 5 Speed', '4 AT', 'at', 'CVT (7 Speed Artificial)', '6 MT',
       'CVT 7 speed', '5 AT', '4 A/T', 'AT 4-speed', 'Manual 5', 'mt',
       '6 AT', 'M/T 5-speed', '6 Speed AT', 'A/t', 'Mt', 'AT 4MATIC',
       'A/T 7 Speed', 'DCT', 'A/T 7 Spd', '5G-Tronic', '5 A/T',
       'AT (7G-Tronic)', '9G-Tronic', 'AT 4 speed', '5 Speed AT',
       '7G-Tronic', '7g-tronic', 'AT 7', '7AT', '9G-Tronik', 'AT 6-spd',
       'Cvt at', 'AT 4-Speed', '6MT', 'Electric',

There are definitely many, many types of transmissions. For the purpose of this demo, I just split them into 3 categories:

* MT: Manual Transmission
* CVT: Continuously Variable (Automatic) Transmission
* AT: Automatic Transmission, this includes every transmission that isn't MT or CVT

In [74]:
df["gearbox_normalized"] = "AT"
df.loc[df["GEARBOX"].isin(["4MT", "5MT", "MT"]), "gearbox_normalized"] = "MT"
df.loc[df["GEARBOX"].isin(["CVT"]), "gearbox_normalized"] = "CVT"
df["gearbox_normalized"].unique()

array(['CVT', 'MT', 'AT'], dtype=object)

Now let's deal with the Tahun (model year) column


In [75]:
df.loc[:, ['Jenis Kendaraan', 'Tahun']].sort_values("Tahun", ascending=False).head()

,Jenis Kendaraan,Tahun
920,Volvo 960 Ex KTT,A
266,Honda HRV SE,2022
145,Honda Brio 1.2 RS CVT,2022
255,Honda HR-V S,2022
254,Honda HR-V RS,2022


A quick search on KTT in Indonesia and Volvo 960, we could conclude the car has the model year of 1992, so let's put 1992 there.
While we are at it, let's also convert it into integers. If there's an unlikely value, it would throw an error.

In [76]:
df.loc[df["Tahun"].isin(["A"]), "Tahun"] = 1992
df["Tahun"] = df["Tahun"].astype("int")

Let's take a look at the fuel consumption column

In [77]:
df["Konsumsi dalam kota (KM/L)"].unique()

array(['8', '8-9', '5-7', '9-10', '12', '12.5', '7', '6', '9', '6.8',
       '11', '10', '10-12', '5-5.5', '6-7', '4-5.5', '8/9', '7.8', '7-8',
       '6.2', '5-6.5', '7-9', '6.8 - 7.2', '5.5-6.5', '6.5-11', '8-10',
       '8,5-9', '10.5', '5.6', '3.8', '4', '3', '5.5-6', '2', '9/10',
       '12-13', '9.8-11.2', '14-15', '16', '14', '14.1', '11-13', '15',
       '13-15', '11-14', '12-14.5', '19.6', '10 - 11', '8-11', '10-11',
       '11-12', '14.7', '5-6', '5', '10.9', '7.3', '6.5', '14-16', '15.1',
       '13', '10.4', '13.3', '11.1', '10.8', '8.5-9', '11-3', '11.5',
       '8.5', '7.5', '9.1', '1:9', '9-11', '7.9', '8-5', '9.0', '8-8',
       '8.5-9.5', '8,2-10,2', '1:7 - 1:8', '8.4', '6.5 - 7', '6,5-7,5',
       '9.5', '9,5', '1-12', '9-10,5', '11-12 / 12-13', '1:9-10', '13-14',
       '12-14', '9.8', '9-12', '11-4', '12.9', '11 - 12', '16-17',
       '12-15', '7-5', '7-9,5', '6-8', '4-6', '11-5', '1:13', '9.5-11',
       '1:10', '8.8', '3,9-4,2', '8,5-9,5', '6,7-8,5', '13.8', '15.8

Turns out it's quite problematic since it is a free text form. For the purpose of this demo, let's just prune every non-numeric values.

In [78]:
df["city"] = pd.to_numeric(df["Konsumsi dalam kota (KM/L)"], errors='coerce')
df["hwy"] = pd.to_numeric(df["Konsumsi luar kota (KM/L)"], errors='coerce')
df.loc[:, ['city', "hwy"]].head()

,city,hwy
0,8.0,12.0
1,NaN,NaN
2,NaN,10.0
3,NaN,15.0
4,12.0,16.0


Now we prune every row that contains a NaN

In [79]:
df = df.dropna(subset=["city"])
df = df.dropna(subset=["hwy"])
df.shape

(550, 32)

We still got 550 rows. We sadly had to remove half of our data. While this is NOT considered a good practice in the industry, for the purpose of this demo, it is still workable.

Let's check our brand spankin new columns!

In [80]:
df[["city", "hwy"]].describe()

,city,hwy
count,550.000000,5.500000e+02
mean,9.472273,2.114865e+05
std,2.749297,4.959484e+06
min,2.000000,4.000000e+00
25%,8.000000,1.100000e+01
50%,9.000000,1.300000e+01
75%,11.000000,1.500000e+01
max,23.000000,1.163102e+08


Whoops, a dude put an unusually large highway MPG, let's just remove that anomaly.

(I checked the data, it's a Suzuki Ertiga. There's NO WAY it's that efficient LOL)

In [81]:
df = df.query("hwy < 100 & city < 100")
df[["city", "hwy"]].describe()

,city,hwy
count,549.000000,549.000000
mean,9.467668,13.351093
std,2.749681,3.364700
min,2.000000,4.000000
25%,8.000000,11.000000
50%,9.000000,13.000000
75%,11.000000,15.000000
max,23.000000,25.000000


Now for the vehicle model, we could not possibly create a database for every possible model out there, there simply isn't enough data to make any meaningful model. In addition, each users may put the same vehicle model in many, many different ways. For the purpose of this demo, we just use the brand of the vehicle.

Let's assume the first word of "Jenis Kendaraan" (Vehicle Type) is the brand, and we just use the vehicle brand to aid our model. After all, an old Cadillac isn't particularly known for its fuel efficiency, right?

In [82]:
df['brand'] = df['Jenis Kendaraan'].str.split().str[0].str.lower()
df["brand"].unique()

array(['audi', 'bmw', 'cadillac', 'chevrolet', 'daihatsu', 'datsun',
       'ford', 'honda', 'hyundai', 'jaguar', 'jeep', 'kia', 'land',
       'lexus', 'mazda', 'mercedes-benz', 'mg', 'mini', 'mitsubishi',
       'morris', 'nissan', 'opel/chevrolet', 'peugeot', 'proton', 'range',
       'renault', 'subaru', 'suzuki', 'timor', 'toyota', 'volvo', 'vw',
       'wuling', 'volkswagen', 'daewoo', 'mercedes', 'grandmax'],
      dtype=object)

Ooops, we still need some fixes here. There are some typos, multiple ways to write the same brand, or simply that people forgot to put their vehicle brand at all.

_Fun fact: Karimun is the name of an island in Indonesia. Suzuki thought that was a better name for Indonesians than Wagon R. Meanwhile, Totota is a city in Liberia. That one's just a typo._

In [83]:
df.loc[df["brand"].isin(["grandmax"]), "brand"] = "daihatsu"
df.loc[df["brand"].isin(["mercedes"]), "brand"] = "mercedes-benz"
df.loc[df["brand"].isin(["opel/chevrolet"]), "brand"] = "chevrolet"
df.loc[df["brand"].isin(["karimun"]), "brand"] = "suzuki"
df.loc[df["brand"].isin(["totota"]), "brand"] = "toyota"
df.loc[df["brand"].isin(["volkswagen"]), "brand"] = "vw"

df["brand"].unique()

array(['audi', 'bmw', 'cadillac', 'chevrolet', 'daihatsu', 'datsun',
       'ford', 'honda', 'hyundai', 'jaguar', 'jeep', 'kia', 'land',
       'lexus', 'mazda', 'mercedes-benz', 'mg', 'mini', 'mitsubishi',
       'morris', 'nissan', 'peugeot', 'proton', 'range', 'renault',
       'subaru', 'suzuki', 'timor', 'toyota', 'volvo', 'vw', 'wuling',
       'daewoo'], dtype=object)

Now, I'd consider the data is clean, ready for training our model

In [84]:
df = df.loc[:, ['Tahun', "brand", "CC mesin", "gearbox_normalized", "city", "hwy"]]
df

,Tahun,brand,CC mesin,gearbox_normalized,city,hwy
0,2011,audi,1800.0,CVT,8.0,12.0
4,2015,bmw,1500.0,AT,12.0,16.0
5,2017,bmw,1500.0,AT,12.5,19.0
6,1997,bmw,1800.0,MT,7.0,11.5
7,1990,bmw,1800.0,MT,6.0,10.0
...,...,...,...,...,...,...
1118,2012,mini,1600.0,AT,9.0,14.0
1119,2001,toyota,3000.0,AT,6.1,7.0
1121,1980,mitsubishi,1300.0,AT,6.0,9.0
1129,1991,bmw,2500.0,AT,5.0,7.0


TODO add correlation matrix https://twitter.com/i/status/1747298352423936072

I intend to use an embedding layer for the brand and gearbox rather than one-hot encoded columns. So, first we need to create the index of all our brands and gearboxes

In [85]:
all_brands_list = df["brand"].unique()
all_gearboxes_list = df["gearbox_normalized"].unique()

print(all_brands_list)
print(all_gearboxes_list)

['audi' 'bmw' 'cadillac' 'chevrolet' 'daihatsu' 'datsun' 'ford' 'honda'
 'hyundai' 'jaguar' 'jeep' 'kia' 'land' 'lexus' 'mazda' 'mercedes-benz'
 'mg' 'mini' 'mitsubishi' 'morris' 'nissan' 'peugeot' 'proton' 'range'
 'renault' 'subaru' 'suzuki' 'timor' 'toyota' 'volvo' 'vw' 'wuling'
 'daewoo']
['CVT' 'AT' 'MT']


And then we add the index to the dataframe

In [86]:
import numpy as np

df['brand_idx'] = df['brand'].apply(lambda x: int(np.where(all_brands_list == x)[0]))
df['gearbox_idx'] = df['gearbox_normalized'].apply(lambda x: int(np.where(all_gearboxes_list == x)[0]))

df.loc[:, ["brand_idx", "brand", "gearbox_idx", "gearbox_normalized"]].tail()

,brand_idx,brand,gearbox_idx,gearbox_normalized
1118,17,mini,1,AT
1119,28,toyota,1,AT
1121,18,mitsubishi,1,AT
1129,1,bmw,1,AT
1132,28,toyota,1,AT


Now we need to normalize the data, so they all lie between 0 and 1. For this, I used the scale between the minimum and maximum values, this is especially useful for model years where cars have only been founded very recently in the grand scheme of things

In [87]:
cc_max = df['CC mesin'].max()
cc_min = df['CC mesin'].min()
print(cc_min, cc_max)

year_max = df['Tahun'].max()
year_min = df['Tahun'].min()
print(year_min, year_max)

fuelcons_max = max(df["city"].max(), df["hwy"].max())
fuelcons_min = min(df["city"].min(), df["hwy"].min())
print(fuelcons_min, fuelcons_max)

df['cc_normalized'] = (df['CC mesin'] - cc_min) / (cc_max - cc_min)
df['year_normalized'] = (df['Tahun'] - year_min) / (year_max - year_min)

df.loc[:, ["Tahun", "CC mesin", "year_normalized", "cc_normalized"]].head()

849.0 7800.0
1961 2022
2.0 25.0


,Tahun,CC mesin,year_normalized,cc_normalized
0,2011,1800.0,0.819672,0.136815
4,2015,1500.0,0.885246,0.093656
5,2017,1500.0,0.918033,0.093656
6,1997,1800.0,0.590164,0.136815
7,1990,1800.0,0.475410,0.136815


Now let's prepare the trainable tensors

In [88]:
import torch
from torch import nn

X_columns = ['brand_idx', 'gearbox_idx'] + ['year_normalized', 'cc_normalized']
X = torch.tensor(df[X_columns].values)
y_city = torch.tensor(df["city"].values)
y_city = y_city.reshape(-1, 1)
y_hwy = torch.tensor(df["hwy"].values)
y_hwy = y_hwy.reshape(-1, 1)
X

tensor([[ 0.0000,  0.0000,  0.8197,  0.1368],
        [ 1.0000,  1.0000,  0.8852,  0.0937],
        [ 1.0000,  1.0000,  0.9180,  0.0937],
        ...,
        [18.0000,  1.0000,  0.3115,  0.0649],
        [ 1.0000,  1.0000,  0.4918,  0.2375],
        [28.0000,  1.0000,  0.4426,  0.0937]], dtype=torch.float64)

Now split them into train and test datasets

In [89]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_city_train, y_city_test, y_hwy_train, y_hwy_test = train_test_split(X, y_city, y_hwy)

Let's formulate the models. As previously mentioned, we will use an embedding layer for the brand and gearbox

In [90]:
class NeuralNetwork(nn.Module):
    def __init__(self, embedding_dim=8):
        super().__init__()
        self.flatten = nn.Flatten()
        self.brand_embedding = nn.Embedding(len(all_brands_list), embedding_dim)
        self.gearbox_embedding = nn.Embedding(len(all_gearboxes_list), embedding_dim)
        self.input_tensor_dim = embedding_dim * 2 + 2  # brand embedding + gearbox embedding + city + hwy

        self.city = nn.Sequential(
            nn.Linear(self.input_tensor_dim, self.input_tensor_dim // 2),
            nn.ReLU(),
            nn.Linear(self.input_tensor_dim // 2, self.input_tensor_dim // 4),
            nn.ReLU(),
            nn.Linear(self.input_tensor_dim // 4, 1)
        )
        self.hwy = nn.Sequential(
            nn.Linear(self.input_tensor_dim, 1),
            nn.Dropout(p=0.3),  # prevent overfitting
        )

    def combine_features(self, x):
        return torch.cat([
            self.brand_embedding(x[:, 0].long()),
            self.gearbox_embedding(x[:, 1].long()),
            x[:, 2].unsqueeze(1),
            x[:, 3].unsqueeze(1),
        ], dim=1)

    def forward(self, x):
        x = self.combine_features(x)
        return self.city(x), self.hwy(x)


device = 'cpu'  # this is a simple model
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (brand_embedding): Embedding(33, 8)
  (gearbox_embedding): Embedding(3, 8)
  (city): Sequential(
    (0): Linear(in_features=18, out_features=9, bias=True)
    (1): ReLU()
    (2): Linear(in_features=9, out_features=4, bias=True)
    (3): ReLU()
    (4): Linear(in_features=4, out_features=1, bias=True)
  )
  (hwy): Sequential(
    (0): Linear(in_features=18, out_features=1, bias=True)
    (1): Dropout(p=0.3, inplace=False)
  )
)


Let's train them

In [91]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=0.1)

X_train = X_train.to(device, dtype=torch.float32)
y_city_train = y_city_train.to(device, dtype=torch.float32)
y_hwy_train = y_hwy_train.to(device, dtype=torch.float32)
X_test = X_test.to(device, dtype=torch.float32)
y_city_test = y_city_test.to(device, dtype=torch.float32)
y_hwy_test = y_hwy_test.to(device, dtype=torch.float32)

for epoch in range(200):
    model.train()  # training mode
    y_city_train_pred, y_hwy_train_pred = model(X_train)
    loss_city = loss_fn(y_city_train_pred, y_city_train)
    loss_hwy = loss_fn(y_hwy_train_pred, y_hwy_train)

    # backward pass
    optimizer.zero_grad()
    loss = loss_city + loss_hwy
    loss.backward()
    optimizer.step()  # update weights

    # evaluate
    model.eval()
    y_city_test_pred, y_hwy_test_pred = model(X_test)
    print(epoch, loss_fn(y_city_test_pred, y_city_test), loss_fn(y_hwy_test_pred, y_hwy_test))

0 tensor(75.2873, grad_fn=<MseLossBackward0>) tensor(175.1271, grad_fn=<MseLossBackward0>)
1 tensor(43.3939, grad_fn=<MseLossBackward0>) tensor(155.3586, grad_fn=<MseLossBackward0>)
2 tensor(18.5766, grad_fn=<MseLossBackward0>) tensor(132.8461, grad_fn=<MseLossBackward0>)
3 tensor(59.1818, grad_fn=<MseLossBackward0>) tensor(108.9806, grad_fn=<MseLossBackward0>)
4 tensor(18.9789, grad_fn=<MseLossBackward0>) tensor(91.0008, grad_fn=<MseLossBackward0>)
5 tensor(8.9608, grad_fn=<MseLossBackward0>) tensor(74.7580, grad_fn=<MseLossBackward0>)
6 tensor(15.8601, grad_fn=<MseLossBackward0>) tensor(59.1951, grad_fn=<MseLossBackward0>)
7 tensor(21.1440, grad_fn=<MseLossBackward0>) tensor(44.7387, grad_fn=<MseLossBackward0>)
8 tensor(21.7655, grad_fn=<MseLossBackward0>) tensor(32.8741, grad_fn=<MseLossBackward0>)
9 tensor(19.0378, grad_fn=<MseLossBackward0>) tensor(24.5135, grad_fn=<MseLossBackward0>)
10 tensor(15.3250, grad_fn=<MseLossBackward0>) tensor(19.3837, grad_fn=<MseLossBackward0>)
11 ten

Just for fun, let's try to make a prediction, just put anything as instructed

In [96]:
input_gearbox = ""
while input_gearbox not in all_gearboxes_list:
    input_gearbox = input("Gearbox: " + str(all_gearboxes_list) + "?")
    input_gearbox = input_gearbox.upper()

input_brand = ""
while input_brand not in all_brands_list:
    input_brand = input("Brand: " + str(all_brands_list) + "?")
    input_brand = input_brand.lower()

input_year = 0
while input_year < year_min or input_year > year_max:
    input_year = input("Model Year (between " + str(year_min) + " and " + str(year_max) + ")")
    input_year = float(input_year)

input_cc = 0
while input_cc < cc_min or input_cc > cc_max:
    input_cc = input("Engine cc (between " + str(cc_min) + " and " + str(cc_max) + ")")
    input_cc = float(input_cc)

X_pred = {
    "brand": input_brand,
    "gearbox_normalized": input_gearbox,
    "brand_idx": int(np.where(all_brands_list == input_brand)[0]),
    "gearbox_idx": int(np.where(all_gearboxes_list == input_gearbox)[0]),
    "year_normalized": float((input_year - year_min) / (year_max - year_min)),
    "cc_normalized": float((input_cc - cc_min) / (cc_max - cc_min)),
}
X_pred = pd.DataFrame(X_pred, index=[0])
df = pd.concat([df, X_pred], axis=0, ignore_index=True)

df.tail()

,Tahun,brand,CC mesin,gearbox_normalized,city,hwy,brand_idx,gearbox_idx,cc_normalized,year_normalized
545,2001.0,toyota,3000.0,AT,6.1,7.0,28,1,0.309452,0.655738
546,1980.0,mitsubishi,1300.0,AT,6.0,9.0,18,1,0.064883,0.311475
547,1991.0,bmw,2500.0,AT,5.0,7.0,1,1,0.237520,0.491803
548,1988.0,toyota,1500.0,AT,10.0,16.0,28,1,0.093656,0.442623
549,NaN,bmw,NaN,CVT,NaN,NaN,1,0,0.007337,0.672131


And here's your prediction!

In [97]:
X_pred = df.iloc[-1]
X_pred = X_pred[X_columns]
X_pred = torch.tensor(X_pred.values.astype(float)).reshape(1, -1)

X_pred = X_pred.to(device, dtype=torch.float32)
y_city_pred, y_hwy_pred = model(X_pred)
print(int(input_year), input_brand, int(input_cc), input_gearbox)
print("City km/L:", float(y_city_pred))
print("Highway km/L:", float(y_hwy_pred))

2002 bmw 900 CVT
City km/L: 10.48488712310791
Highway km/L: 9.835464477539062
